# Análisis de Datos - Premier League

En este notebook se analiza un dataset con estadísticas históricas de jugadores de la Premier League con el objetivo de predecir su rendimiento. Se realizará una limpieza de datos, análisis exploratorio y cálculo de estadísticas clave como valores nulos, ceros y covarianzas.

Además, se identifican columnas poco relevantes que podrían eliminarse para facilitar futuros modelos predictivos.



In [1]:
import numpy as np
import pandas as pd

print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)


NumPy version: 2.2.4
Pandas version: 2.2.3


In [2]:
import pandas as pd

# Abre el archivo y salta las líneas hasta encontrar @DATA
with open("Premiere-League_Stats.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

# Encuentra el índice donde comienzan los datos
data_index = next(i for i, line in enumerate(lines) if line.strip().upper() == "@DATA")

# Lee solo los datos, ignorando la cabecera ARFF
data_lines = lines[data_index + 1:]

# Convierte en DataFrame
from io import StringIO

df = pd.read_csv(StringIO("".join(data_lines)), header=None)


In [5]:
df.columns = [
    "Nombre", "Número_Camiseta", "Club", "Posición", "Nacionalidad", "Edad", "Apariciones", "Victorias", "Derrotas", "Goles",
    "Goles_por_partido", "Goles_de_cabeza", "Goles_con_pie_derecho", "Goles_con_pie_izquierdo", "Penales_anotados",
    "Tiros_libres_anotados", "Disparos", "Disparos_al_arco", "Precisión_disparo_%", "Tiros_al_palo", "Ocasiones_claras_falladas",
    "Porterías_a_cero", "Goles_contra", "Entradas", "Éxito_entradas_%", "Entradas_como_último_hombre", "Disparos_bloqueados",
    "Intercepciones", "Despejes", "Despejes_de_cabeza", "Despejes_sobre_la_línea", "Recuperaciones", "Duelos_ganados",
    "Duelos_perdidos", "Duelos_50_50_exitosos", "Duelos_aéreos_ganados", "Duelos_aéreos_perdidos", "Autogoles",
    "Errores_que_generan_gol", "Asistencias", "Pases", "Pases_por_partido", "Ocasiones_claras_creadas", "Centros",
    "Precisión_centros_%", "Pases_filtrados", "Pases_largos_acertados", "Atajadas", "Penales_atajados", "Despejes_con_los_puños",
    "Cortes_altos", "Atajadas_con_las_manos", "Cierres_como_líbero", "Saque_con_las_manos", "Saque_de_meta", "Tarjetas_amarillas",
    "Tarjetas_rojas", "Faltas", "Fueras_de_juego"
]




In [6]:
import numpy as np

# Reemplazar los '?' por valores NaN
df.replace('?', np.nan, inplace=True)

# Convertir todas las columnas a numérico (las que puedan)
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')


## Conteo de valores nulos y ceros

Antes de continuar con el análisis, es importante revisar la calidad del dataset. A continuación, se muestra el conteo de:

- **Valores nulos (NaN)** por columna: indican datos faltantes que podrían afectar el análisis o los modelos.
- **Valores en cero (0)** por columna: pueden representar ausencia de una acción (por ejemplo, 0 goles) o poca participación del jugador en ciertos aspectos del juego.

Este paso permite identificar columnas que podrían ser eliminadas por tener demasiados valores faltantes o por no aportar información significativa.


In [ ]:
# Conteo de valores nulos
print("Valores nulos por columna:\n")
print(df.isnull().sum())

# Conteo de ceros
print("\nCantidad de ceros por columna:\n")
print((df == 0).sum())

##  Eliminación de columnas irrelevantes

Con base en el análisis de valores nulos y ceros, se eliminaron varias columnas que presentan problemas de calidad o baja relevancia para el análisis general del rendimiento de los jugadores.

###  Criterios utilizados:

- Columnas con **más del 50% de valores nulos** (más de 250).
- Columnas con **baja variabilidad o demasiados ceros**, lo que indica poca utilidad en el modelo.
- Columnas específicas para **porteros**, que no aplican a la mayoría de los jugadores de campo.
- Columnas **categóricas no numéricas** (`Nombre`, `Club`, `Posición`, `Nacionalidad`), que no se usarán en este análisis.

###  Columnas eliminadas:

- **Identificadores y datos no numéricos:** `Nombre`, `Número_Camiseta`, `Club`, `Posición`, `Nacionalidad`
- **Columnas con más del 50% de valores nulos:** 
  - `Goles_por_partido`, `Penales_anotados`, `Tiros_libres_anotados`, `Disparos`, `Disparos_al_arco`
  - `Precisión_disparo_%`, `Ocasiones_claras_falladas`, `Porterías_a_cero`, `Goles_contra`
  - `Entradas_como_último_hombre`, `Despejes_sobre_la_línea`, `Precisión_centros_%`, `Pases_filtrados`
  - `Pases_largos_acertados`, `Atajadas`, `Penales_atajados`, `Despejes_con_los_puños`, `Cortes_altos`
  - `Atajadas_con_las_manos`, `Cierres_como_líbero`, `Saque_con_las_manos`, `Saque_de_meta`
- **Otras con valores poco informativos o muy constantes:** `Autogoles`, `Errores_que_generan_gol`, `Tarjetas_rojas`

Estas columnas fueron eliminadas para mejorar la calidad del dataset y dejar únicamente variables relevantes y generalizables para modelos predictivos.


In [8]:
columnas_a_eliminar = [
    "Nombre", "Número_Camiseta", "Club", "Posición", "Nacionalidad",
    "Goles_por_partido", "Penales_anotados", "Tiros_libres_anotados", "Disparos", "Disparos_al_arco",
    "Precisión_disparo_%", "Ocasiones_claras_falladas", "Porterías_a_cero", "Goles_contra",
    "Entradas_como_último_hombre", "Despejes_sobre_la_línea", "Precisión_centros_%", "Pases_filtrados",
    "Pases_largos_acertados", "Atajadas", "Penales_atajados", "Despejes_con_los_puños", "Cortes_altos",
    "Atajadas_con_las_manos", "Cierres_como_líbero", "Saque_con_las_manos", "Saque_de_meta",
    "Autogoles", "Errores_que_generan_gol", "Tarjetas_rojas"
]

# Eliminar solo si la columna existe (para evitar errores)
df = df.drop(columns=[col for col in columnas_a_eliminar if col in df.columns])



## Varianza por columna

La varianza mide la dispersión de los datos en cada columna numérica. Una varianza alta indica que los valores están muy separados entre sí; una varianza baja indica que los valores están más agrupados.

Este análisis permite identificar variables que pueden tener poca variación y, por lo tanto, menor relevancia predictiva. No es obligatorio eliminar columnas con baja varianza, pero puede ser útil para simplificar modelos.


In [9]:
# Calcular la varianza de todas las columnas numéricas
varianzas = df.var(numeric_only=True)

# Mostrar la varianza de cada columna
print("Varianza por columna:\n")
print(varianzas)


Varianza por columna:

Edad                        1.916122e+01
Apariciones                 6.690834e+03
Victorias                   1.518612e+03
Derrotas                    8.115876e+02
Goles                       2.825926e+02
Goles_de_cabeza             1.178806e+01
Goles_con_pie_derecho       1.220658e+02
Goles_con_pie_izquierdo     3.963524e+01
Tiros_al_palo               1.506804e+01
Entradas                    2.006365e+04
Éxito_entradas_%            7.812339e+02
Disparos_bloqueados         1.045501e+03
Intercepciones              1.191738e+04
Despejes                    9.391794e+04
Despejes_de_cabeza          2.065401e+04
Recuperaciones              1.634164e+05
Duelos_ganados              1.508059e+05
Duelos_perdidos             1.165941e+05
Duelos_50_50_exitosos       5.531624e+03
Duelos_aéreos_ganados       2.493802e+04
Duelos_aéreos_perdidos      1.204544e+04
Asistencias                 9.529205e+01
Pases                       9.779546e+06
Pases_por_partido           3.4473

##  Análisis de varianza por columna

La varianza mide la dispersión de los valores en una variable. Cuanto mayor es la varianza, más diversa es la distribución de los datos en esa columna. Este análisis permite detectar variables con **poca variabilidad**, que podrían aportar **menos información** a modelos predictivos, así como identificar aquellas con **alta varianza**, que pueden ser más significativas.

###  Columnas con mayor varianza
Las siguientes variables muestran una alta dispersión entre jugadores, lo que indica diferencias notables en el rendimiento o participación. Son ideales para conservar en el análisis:

- **Pases** (`~9.7 millones`): enorme varianza, refleja grandes diferencias en participación entre jugadores.
- **Recuperaciones**, **Duelos ganados**, **Duelos perdidos**: reflejan el impacto defensivo.
- **Despejes** y **Centros**: tareas específicas en jugadores defensivos y de banda.

Estas columnas son útiles para modelar roles y rendimiento según la posición y comportamiento en el juego.

---

###  Columnas con menor varianza
Las siguientes columnas muestran valores más agrupados o concentrados, por lo que podrían aportar menos información al modelo:

- **Edad**: la mayoría de los jugadores está en un rango similar.
- **Goles de cabeza**, **Tiros al palo**, **Asistencias**, **Tarjetas amarillas**: varían poco entre jugadores.
- **Goles** y **Goles con pie izquierdo o derecho**: aunque aportan valor ofensivo, su variabilidad es más limitada.

Estas columnas **no necesariamente deben eliminarse**, pero podrían analizarse más a fondo si se busca optimizar un modelo predictivo reduciendo la cantidad de variables.





##  Conclusión general del análisis

En este notebook se realizó un análisis exploratorio completo del dataset de estadísticas históricas de jugadores de la Premier League, con el propósito de preparar los datos para futuros modelos de predicción del rendimiento.

###  Resumen de pasos realizados:

-  Se cargó el dataset y se identificaron **valores nulos** y **valores en cero** por columna, lo que permitió detectar posibles problemas de calidad en los datos.
-  Se eliminaron columnas **irrelevantes o con más del 50% de valores nulos**, incluyendo estadísticas específicas para porteros y datos categóricos no útiles para el análisis numérico.
-  Se realizó un análisis de **varianza**, que permitió identificar qué columnas presentan mayor dispersión de datos y por tanto podrían tener un mayor peso explicativo en modelos predictivos.
-  Se analizaron variables con **baja varianza**, que si bien no fueron eliminadas automáticamente, podrían ser descartadas si se decide reducir la complejidad del modelo.

###  Resultado

El dataset final ha sido depurado y contiene variables numéricas relevantes, limpias y adecuadas para análisis estadístico y desarrollo de modelos de machine learning orientados a predecir el rendimiento de los jugadores.

Este trabajo sienta una base sólida para construir modelos que aprovechen datos históricos reales de la Premier League y generen información útil para scouting, análisis de desempeño y toma de decisiones estratégicas en el fútbol.
